In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
from tensorflow import keras
import tensorflow as tf
import warnings
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import cross_val_score

from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectPercentile
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
plt.rc('font', family=fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()) # for Windows OS user
import datetime
import os

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge
from tensorflow import keras
from scipy.stats.mstats import gmean
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")

In [3]:
import shap

In [4]:
y = pd.read_csv(os.path.abspath("../input")+'/y_train.csv', encoding='cp949')
trn = pd.read_csv(os.path.abspath("../input")+'/X_train.csv', encoding='cp949')
tst = pd.read_csv(os.path.abspath("../input")+'/X_test.csv', encoding='cp949')
IDtrain = trn.custid.unique()
IDtest = tst.custid.unique()

In [5]:
# 1등팀 features
import pickle

with open('../features/X_train_select_1st.pkl', 'rb') as f:
    X_train = pickle.load(f)
    
with open('../features/X_test_select_1st.pkl', 'rb') as f:
    X_test = pickle.load(f)
    
X_train1 = pd.DataFrame(X_train)
X_test1 = pd.DataFrame(X_test)


In [6]:
# 2등팀 feature engineering 참고
X_train2 = pd.read_csv(os.path.abspath("../features")+'/X_train_select_2nd.csv', encoding='cp949')
X_test2 = pd.read_csv(os.path.abspath("../features")+'/X_test_select_2nd.csv', encoding='cp949')


In [7]:
# 3등팀 feature engineering 참고
X_train3 = pd.read_csv(os.path.abspath("../features")+'/X_train_select_3rd.csv', encoding='cp949')
X_test3 = pd.read_csv(os.path.abspath("../features")+'/X_test_select_3rd.csv', encoding='cp949')

X_train3 = X_train3.iloc[:,2:]
X_test3 = X_test3.iloc[:,2:]

In [8]:
X_train1['custid'] = IDtrain
X_train2['custid'] = IDtrain
X_train3['custid'] = IDtrain

X_test1['custid'] = IDtest
X_test2['custid'] = IDtest
X_test3['custid'] = IDtest

In [9]:
display(X_train1.head())
display(X_test1.head())
print(X_train1.shape, X_test1.shape, y.shape)

display(X_train2.head())
display(X_test2.head())
print(X_train2.shape, X_test2.shape, y.shape)

display(X_train3.head())
display(X_test3.head())
print(X_train3.shape, X_test3.shape, y.shape)

,0,1,2,3,4,5,6,7,8,9,...,895,896,897,898,899,900,901,902,903,custid
0,-0.129259,0.522979,0.984931,0.321857,0.453497,1.267911,-0.355535,-0.565138,-0.201601,0.668142,...,0.026032,0.166440,-0.011775,0.277593,0.254895,0.080834,0.023582,0.211609,-0.298411,0
1,-0.950020,0.455549,2.119481,-1.662230,1.959637,2.141412,-0.248856,4.152913,-0.201601,3.147699,...,-0.031413,-0.120867,0.114782,0.281708,0.186661,0.215307,-0.065986,0.088980,-0.250751,2
2,-0.630835,0.610638,1.087506,-0.578141,0.122238,1.059934,0.391222,0.409431,0.095918,0.127219,...,-0.371944,-0.152953,-0.046666,0.153409,-0.015751,0.352490,-0.273290,-0.253228,-0.095880,3
3,0.838428,-2.713662,-2.247293,0.335493,-0.158587,3.576451,-0.553655,5.200848,1.498509,0.336641,...,-0.003926,0.063787,0.150303,0.357579,0.281164,0.204657,0.096526,0.094631,-0.198050,4
4,-0.696699,0.401605,1.616230,0.296404,0.973883,-0.026744,0.086423,0.114032,-0.108626,1.027017,...,0.024427,-0.085707,0.070068,0.346216,0.243764,0.061486,0.119414,0.132112,-0.273083,5


,0,1,2,3,4,5,6,7,8,9,...,895,896,897,898,899,900,901,902,903,custid
0,2.054371,0.388119,-1.644922,0.180448,0.341167,-0.211034,-0.111696,0.074077,0.113234,0.270940,...,0.155051,-0.010222,0.074419,0.330232,0.271155,0.140621,-0.009257,0.111533,-0.341997,30001
1,-0.950020,0.522979,0.717249,-0.290415,-0.119615,0.225581,2.372414,0.105926,-1.251669,-0.215636,...,-0.027063,-0.143582,0.099644,0.288969,0.234549,0.122477,0.100111,0.114257,-0.259047,30002
2,0.027800,0.522979,0.430884,0.080493,-0.431388,-0.760525,0.132143,-0.302292,-0.391507,-0.578317,...,-0.204982,0.200916,-0.057595,0.248941,0.407649,-0.005065,-0.242158,-0.194146,-0.253173,30003
3,3.371641,0.340918,-2.525086,0.346402,-0.524232,1.517482,-0.782254,-1.576361,1.498509,-0.714367,...,-0.163169,0.448237,0.075410,0.171209,0.478824,-0.071823,-0.404565,-0.172041,0.232815,30005
4,1.532529,0.543208,-0.429491,0.308220,-0.515062,-0.770260,-0.706054,-0.207158,1.498509,-0.547599,...,-0.111255,0.022932,-0.041291,0.243981,0.351951,0.090851,-0.074801,0.014036,-0.299642,30007


(21587, 905) (14380, 905) (21587, 2)


,0,1,2,3,4,5,6,7,8,9,...,673,674,675,676,677,678,679,680,681,custid
0,-0.277038,-0.413497,-0.091169,-0.616368,-0.651589,-0.679368,-0.698617,-0.697065,0.088745,0.545715,...,0.186348,-0.137314,0.023652,-0.071803,-0.023077,0.015955,0.067195,0.123257,0.017069,0
1,0.378278,-0.190388,-0.525381,-0.387562,-0.436070,-0.679368,-0.504273,-0.497683,-0.468103,-0.093352,...,0.116229,-0.165716,-0.097732,-0.213580,-0.238919,0.138185,0.029009,-0.020097,-0.129550,2
2,-0.277038,0.032722,0.125937,-0.158756,-0.005032,-0.679368,-0.698617,0.299844,1.388057,-0.093352,...,0.041655,0.041977,0.170166,-0.128733,0.137020,0.347108,0.067634,-0.038666,0.182087,3
3,-0.277038,-0.413497,-0.525381,-0.616368,0.210487,-0.679368,-0.698617,-0.697065,-0.653719,-0.519396,...,0.422866,-0.077372,-0.245819,-0.099829,-0.103595,0.228107,-0.118550,0.429022,-0.331686,4
4,-0.277038,1.371377,-0.308275,-0.158756,-0.651589,-0.482155,-0.115587,0.299844,0.459977,0.758737,...,0.057249,-0.026505,0.097697,-0.138563,0.078147,0.075115,0.143055,0.022794,0.030847,5


,0,1,2,3,4,5,6,7,8,9,...,673,674,675,676,677,678,679,680,681,custid
0,-0.277038,-0.413497,-0.525381,-0.616368,-0.651589,-0.679368,-0.115587,2.293663,0.645593,-0.306374,...,0.014752,-0.168588,-0.010406,-0.157667,0.112811,0.047409,0.072320,0.115210,0.150447,30001
1,0.378278,2.040705,3.382527,1.900499,1.072563,1.095545,1.050473,2.293663,0.645593,0.545715,...,0.144448,-0.022424,0.071430,-0.131902,-0.028585,-0.037755,0.016143,0.086632,0.058499,30002
2,-0.277038,-0.413497,0.343043,-0.158756,1.934638,-0.284943,-0.504273,1.097372,0.831209,0.545715,...,-0.010721,-0.125940,0.054630,-0.135784,0.157583,0.156054,0.078817,0.130739,-0.159531,30003
3,-0.277038,-0.413497,-0.525381,-0.616368,-0.651589,-0.679368,-0.698617,-0.697065,-0.653719,-0.306374,...,0.071522,-0.125335,-0.176496,-0.003760,-0.037680,-0.109866,0.056316,0.126365,0.134055,30005
4,-0.277038,-0.413497,-0.525381,-0.616368,-0.651589,-0.482155,-0.504273,-0.697065,-0.282487,-0.093352,...,0.131548,-0.087861,0.014829,-0.097195,0.031313,-0.035392,-0.047617,0.261103,-0.035998,30007


(21587, 683) (14380, 683) (21587, 2)


,0,1,2,3,4,5,6,7,8,9,...,799,800,801,802,803,804,805,806,807,custid
0,14.370545,2.484907,11.972655,13.028055,12.067965,14.265185,11.867296,1.339774,1.386294,0.492476,...,0.020167,0.052756,0.008474,0.062249,0.065455,0.056593,-0.004349,-0.009070,-0.051147,0
1,15.137493,2.484907,12.368329,14.163347,12.449376,15.067062,12.330205,1.386294,2.197225,0.492476,...,0.059626,0.016700,-0.012409,0.046722,0.048045,0.069787,-0.022880,0.016665,-0.063029,2
2,14.648637,3.433987,11.247453,13.339088,11.425852,14.607979,11.206794,1.299283,1.791759,0.530628,...,0.034221,0.052203,-0.006304,0.012237,0.037133,0.063931,-0.022545,-0.007070,-0.043196,3
3,13.859528,1.609438,12.368329,13.235694,9.989711,13.838447,12.330205,1.386294,2.197225,0.693147,...,-0.048252,0.072581,-0.023287,0.014033,-0.019005,0.062262,0.006878,0.026838,-0.059535,4
4,15.435643,3.496508,11.969913,13.432786,12.797356,15.361476,11.895747,1.056053,1.386294,0.504556,...,-0.005587,0.033537,0.015377,0.002062,0.017813,0.029902,-0.023758,0.010486,-0.022652,5


,0,1,2,3,4,5,6,7,8,9,...,799,800,801,802,803,804,805,806,807,custid
0,15.240622,3.332205,11.944791,13.607257,11.679041,15.211817,11.915987,1.008228,1.386294,0.532805,...,-0.020609,0.071663,0.004754,0.032206,0.017161,0.005254,-0.018671,0.034539,-0.012176,30001
1,15.903696,4.543295,11.278736,12.923915,12.660490,15.863875,11.238915,1.118030,1.386294,0.344840,...,0.003736,0.050479,-0.011731,0.028123,0.034544,0.035079,-0.025735,-0.008565,-0.036181,30002
2,14.147416,3.871201,10.316067,11.835016,9.922849,14.132677,10.287882,0.850333,1.386294,0.467341,...,0.029580,0.052457,-0.024625,0.001446,0.036565,0.001737,-0.016135,0.023261,-0.009722,30003
3,11.790081,1.098612,10.950824,11.141876,8.160804,11.746819,10.899532,1.386294,1.386294,0.693147,...,0.017756,0.098537,0.007508,-0.040759,0.041574,-0.023873,0.016691,0.015720,-0.122590,30005
4,12.596427,1.945910,10.804685,11.141876,9.220390,12.561650,10.769909,0.847298,1.386294,0.693147,...,0.017681,0.045315,-0.038612,0.055294,0.083943,0.048235,0.008936,-0.010461,-0.040079,30007


(21587, 809) (14380, 809) (21587, 2)


In [13]:
# 1,2,3등팀 feature 병합
X_train = X_train1.merge(X_train2)
X_test = X_test1.merge(X_test2)

In [14]:
display(X_train.head())
display(X_test.head())
print(X_train.shape, X_test.shape, y.shape)

,0,1,2,3,4,5,6,7,8,9,...,672,673,674,675,676,677,678,679,680,681
0,-0.129259,0.522979,0.984931,0.321857,0.453497,1.267911,-0.355535,-0.565138,-0.201601,0.668142,...,0.315908,0.186348,-0.137314,0.023652,-0.071803,-0.023077,0.015955,0.067195,0.123257,0.017069
1,-0.950020,0.455549,2.119481,-1.662230,1.959637,2.141412,-0.248856,4.152913,-0.201601,3.147699,...,0.508378,0.116229,-0.165716,-0.097732,-0.213580,-0.238919,0.138185,0.029009,-0.020097,-0.129550
2,-0.630835,0.610638,1.087506,-0.578141,0.122238,1.059934,0.391222,0.409431,0.095918,0.127219,...,0.189670,0.041655,0.041977,0.170166,-0.128733,0.137020,0.347108,0.067634,-0.038666,0.182087
3,0.838428,-2.713662,-2.247293,0.335493,-0.158587,3.576451,-0.553655,5.200848,1.498509,0.336641,...,-0.030774,0.422866,-0.077372,-0.245819,-0.099829,-0.103595,0.228107,-0.118550,0.429022,-0.331686
4,-0.696699,0.401605,1.616230,0.296404,0.973883,-0.026744,0.086423,0.114032,-0.108626,1.027017,...,0.240199,0.057249,-0.026505,0.097697,-0.138563,0.078147,0.075115,0.143055,0.022794,0.030847


,0,1,2,3,4,5,6,7,8,9,...,672,673,674,675,676,677,678,679,680,681
0,2.054371,0.388119,-1.644922,0.180448,0.341167,-0.211034,-0.111696,0.074077,0.113234,0.270940,...,0.444342,0.014752,-0.168588,-0.010406,-0.157667,0.112811,0.047409,0.072320,0.115210,0.150447
1,-0.950020,0.522979,0.717249,-0.290415,-0.119615,0.225581,2.372414,0.105926,-1.251669,-0.215636,...,0.298328,0.144448,-0.022424,0.071430,-0.131902,-0.028585,-0.037755,0.016143,0.086632,0.058499
2,0.027800,0.522979,0.430884,0.080493,-0.431388,-0.760525,0.132143,-0.302292,-0.391507,-0.578317,...,0.262527,-0.010721,-0.125940,0.054630,-0.135784,0.157583,0.156054,0.078817,0.130739,-0.159531
3,3.371641,0.340918,-2.525086,0.346402,-0.524232,1.517482,-0.782254,-1.576361,1.498509,-0.714367,...,0.230590,0.071522,-0.125335,-0.176496,-0.003760,-0.037680,-0.109866,0.056316,0.126365,0.134055
4,1.532529,0.543208,-0.429491,0.308220,-0.515062,-0.770260,-0.706054,-0.207158,1.498509,-0.547599,...,0.142403,0.131548,-0.087861,0.014829,-0.097195,0.031313,-0.035392,-0.047617,0.261103,-0.035998


(21587, 1587) (14380, 1587) (21587, 2)


In [22]:
new_col = [a for a,b in enumerate(X_train.columns)]
X_train.columns = new_col
X_test.columns = new_col

In [24]:
# Method 2: Using SHAP values
y_train = y.copy()
# DF, based on which importance is checked
X_importance = X_test

# Explain model predictions using shap library:
model = LGBMRegressor(random_state=0).fit(X_train, y_train.age)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_importance)

# Plot summary_plot as barplot:
#shap.summary_plot(shap_values, X_importance, plot_type='bar')

shap_sum = np.abs(shap_values).mean(axis=0)
importance_df = pd.DataFrame([X_importance.columns.tolist(), shap_sum.tolist()]).T
importance_df.columns = ['column_name', 'shap_importance']
importance_df = importance_df.sort_values('shap_importance', ascending=False)
importance_df

,column_name,shap_importance
651,651.0,2.449043
654,654.0,0.634070
919,919.0,0.534083
652,652.0,0.490592
653,653.0,0.358699
...,...,...
1153,1153.0,0.000000
1154,1154.0,0.000000
1155,1155.0,0.000000
1156,1156.0,0.000000


In [25]:
# 과적합을 피하기 위해 최적의 p값 주변의 값을 선택하는게 더 나은 결과를 얻을 수 있다. 
features_selected = importance_df.query('shap_importance > 0').column_name
X_train = X_train[features_selected]
X_train['custid'] = IDtrain
X_test = X_test[features_selected]
X_test['custid'] = IDtest
print(X_train.shape, X_test.shape)

(21587, 981) (14380, 981)


In [26]:
# 모델링 준비를 위한 파일 내보내기
X_train.to_csv("../features/X_train_shap_merge.csv", index=False)
X_test.to_csv("../features/X_test_shap_merge.csv", index=False)

## END